# Classification using Decision Trees

Decision trees are among the most popular classical machine learning algorithms, which can used for both classification (so-called classification trees) and regression (so-called regression trees). In this notebook, we will use decision trees for classification. 

Our implementation will be based on the open source machine learning library [scikit-learn](https://scikit-learn.org/stable/index.html). For simplicity and interpretability, we will use some synthetic 2D data in this notebook. In [regression_decision_tree.ipynb](regression_decision_tree.ipynb), we will use some scientific data to learn regression with decision trees.

In [ ]:
# sklearn
from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import sklearn.datasets

# helpers
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

---

# Data generation

We start by generating a dataset using the `make_blobs` method from `sklearn`. This method can generate multi-dimensional random data points (blobs) in a Gaussian distribution around a specific number of centres.

In the following cell, we generate a dataset of 300 samples in 2D (2 features) with 4 centres (four classes). The argument `cluster_std` controls the dispersion of the dataset: a larger `cluster_std` makes the dataset more challenging to classify. Change `cluster_std` to see the effect.

In [ ]:
# generate Gaussian blobs
X, y = make_blobs(n_samples=300, n_features=2, centers=4,
                  cluster_std=1.5, random_state=0)

# plot data points
plt.figure(dpi=100)
scat = plt.scatter(X[:, 0], X[:, 1], c=y, s=20, alpha=0.7, edgecolors='k')
plt.xlabel('Feature 0')
plt.ylabel('Feature 1')
plt.gca().add_artist(plt.legend(*scat.legend_elements(), 
                                title='Classes', bbox_to_anchor=(1.2, 1.)))
plt.show()

---

# Decision Tree

## 1. Create a decision tree classifier

Now we create a `DecisionTreeClassifier` object from `sklearn` to classify the blobs. There are many hyperparameters that can be set for the classifier. We will explain some important ones below. For a full list see [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

* **`max_depth`**

The maximum depth of the tree. If left as `None`, nodes are expanded until all leaves are pure or until all leaves contain fewer than `min_samples_split` samples. Here we use `None`, the default value.

* **`min_samples_split`**

The minimum number of samples required to split an internal node. Here we use `min_samples_split=2`, the default value, so our tree will be a binary tree.
    
* **`max_features`**

The maximum number of features considered for splitting. If left as `None`, the classifier will use `n_features` of the input data. Here we use `None`, the default value, so 4

In [ ]:
# create decision tree
decision_tree = DecisionTreeClassifier(max_depth=None, min_samples_split=2, max_features=None)

## 2. Fit the model to data

In this instance, we do not need to do much feature engineering because our data are relative simple with only two features and four classes. The `DecisionTreeClassifier` class has a `fit()` method, which takes the feature `X` and label `y` as inputs and fits the model to the data:

In [ ]:
# fit decision tree
tree = decision_tree.fit(X, y)

The following helper function can be used to visualise the result of the classification. It plots the training data points over a contour plot of model predictions over the entire space.

In [ ]:
# function to visualise classification
def visualize_classifier(X, y, model, ax=None, cmap='viridis'):
    ax = ax or plt.gca()
    
    # plot the training points
    scat = ax.scatter(X[:, 0], X[:, 1], c=y, s=20, alpha=0.7, edgecolors='k',
                      clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create a structured grid of the entire space
    x0, x1 = np.meshgrid(np.linspace(*xlim, num=200), np.linspace(*ylim, num=200))
    
    # compute model predictions on the grid
    y_pred = model.predict(np.c_[x0.ravel(), x1.ravel()]).reshape(x0.shape)

    # create a contour plot of the grid
    n_classes = len(np.unique(y))
    ax.contourf(x0, x1, y_pred, alpha=0.3, levels=np.arange(n_classes + 1) - 0.5, cmap=cmap, zorder=1)
    ax.set(xlim=xlim, ylim=ylim)

In [ ]:
# visualise classification
plt.figure(dpi=100)
visualize_classifier(X, y, tree)
plt.show()

Notice that the classification looks rather arbitrary in the central region. To verify the robustness of the model, let us try re-running `fit()` with some random sub-sets of the training data:

In [ ]:
# number of subsets
n_subsets = 6

# number of points in a subset
n_points_subset = 200

# fit with subsets and plot
fig, axs = plt.subplots(n_subsets // 3, 3, figsize=(13, n_subsets), dpi=100)
for ax in axs.flatten():
    # randomly choose data
    sub_indices = np.random.choice(np.arange(len(y)), n_points_subset)
    # fit and plot
    tree = decision_tree.fit(X[sub_indices], y[sub_indices])
    visualize_classifier(X[sub_indices], y[sub_indices], tree, ax=ax)
plt.tight_layout()
plt.show()

Clearly, our classification is highly inconsistent in the central region where the different classes mix. It is concerning that the details of the classification varies too drastically with the training data. This is a classic sign that the model is **overfitting**.

In this example, we can see visually that overfitting is occurring, but is there a systematic way of detecting overfitting? For this purpose, we first need a **metric** as an objective measurement of how well the model is performing. 

## 3. Model evaluation
    
We can use the `accuracy_score` metric from `sklearn` to quantify the accuracy of the model on this data. Before computing `accuracy_score`, we first re-fit the model with the original `(X, y)` and make the predictions. This time, we are also returning the centres when generating the blobs, which are required later to generate the test dataset. 

In [ ]:
# generate Gaussian blobs
centers= np.random.RandomState(0).uniform(-10, 10, size=(4, 2))
X, y = make_blobs(n_samples=300, n_features=2, centers=centers,
                           cluster_std=1.5, random_state=0)

# re-fit model with (X, y)
tree = decision_tree.fit(X, y)

# make predictions
y_pred = tree.predict(X)

# compute accuarcy score
print('Accuracy score on training data: %.2f' % accuracy_score(y, y_pred))

Pretty accurate! Wait up a moment -- let us try the model on some test data, generated with the same distribution parameters (including the centres) but a different `random_state` (giving different point locations):

In [ ]:
# generate Gaussian blobs
test_X, test_y = make_blobs(n_samples=300, n_features=2, centers=centers,
                            cluster_std=1.5, random_state=1)

# plot test data
plt.figure(dpi=100)
scat = plt.scatter(test_X[:, 0], test_X[:, 1], c=test_y, s=20, alpha=0.7, edgecolors='k')
plt.xlabel('Feature 0')
plt.ylabel('Feature 1')
plt.gca().add_artist(plt.legend(*scat.legend_elements(), 
                                title='Classes', bbox_to_anchor=(1.2, 1.)))
plt.show()

# make predictions with test data
test_y_pred = tree.predict(test_X)

# compute accuarcy score with test data
print('Accuracy score on test data: %.2f' % accuracy_score(test_y, test_y_pred))

That's not quite as good as before! The model is overfitting: it is performing well on the training set but not generalising beyond it.

## 4. Hyperparameter tuning and cross-validation

Overfitting often results when the model has too many parameters and thus fits too exactly to the training set. To alleviate overfitting, we must choose a proper number of parameters in the model. For the `DecisionTreeClassifier` class, it is controlled by the `max_depth` argument. We are going to use a training/test split and cross-validation to tune this hyperparameter. The same method can be applied to the other hyperparameters. 

Below we set up a search across `max_depth` ranging from 1 to 20 by using the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) method. For each value, we use a 5-fold cross-validation.

In [ ]:
# set up a parameter grid to search
# here we only search max_depth
param_grid = {'max_depth': range(1, 20)}

# decision tree object
decision_tree = DecisionTreeClassifier()

# assoicte the gird with the object
search = GridSearchCV(decision_tree, param_grid, cv=5)

# perform search
gs = search.fit(X, y)

In [ ]:
# plot the search result
plt.figure(dpi=100)
plt.plot(search.cv_results_['mean_test_score'])
plt.xlabel('Tree depth')
plt.ylabel('Val Accuracy')
plt.xticks(range(1, 20, 2))
plt.show()

We can see that the cross-validation score reaches its maximum at `max_depth=3`. We take the best performing hyperparameter and retrain the model on the full dataset. Then we can apply this tuned model to the test data and see if the accuracy improves.

In [ ]:
# get the best performing hyperparameter
best_max_depth = gs.best_params_['max_depth']
print('Best max_depth: %d' % best_max_depth)

# create and fit model with the best performing hyperparameter
tree = DecisionTreeClassifier(max_depth=best_max_depth).fit(X, y)

# make predictions with test data
test_y_pred = tree.predict(test_X)

# compute accuarcy score with test data
print('Accuracy score on test data: %.2f' % accuracy_score(test_y, test_y_pred))

## 5. Bagging and boosting

We can improve the performance of decision trees in two ways, *bagging* and *boosting*, both belonging to ensemble learning algorithms. "**Ensemble**" means that the algorithm will plant not just one tree but an ensemble of trees. 

**Bagging** (or bootstrap aggregation) reduces the variance of a decision tree by training an ensemble of trees on random subsets of the training data (with some replacement) and taking the average of the ensemble as the final result. A **Random Forest** is an example of bagging: in addition to a random subset of data, a random forest uses a random subset of features to grow each tree in the ensemble. 

**Boosting**, as another ensemble technique, grows the trees in a sequential manner, that is, the misclassified samples or errors in the prior tree will be fed into the current tree and so forth; at the end, the final result is determined by majority voting from the ensemble. **AdaBoost** and **Gradient Boosting** are the most popular boosting algorithms. From a data perspective, the data points are weighted equally in bagging and unequally in boosting, as the former is parallel and the latter sequential. 

Here we will briefly demonstrate the usage of `RandomForestClassifier`  and `AdaBoostClassifier` from `sklearn`. Gradient boosting will be used for regression in [regression_decision_tree.ipynb](regression_decision_tree.ipynb). 

In [ ]:
# create and fit a random forest classifier
forest = RandomForestClassifier(n_estimators=300, random_state=0).fit(X, y)

# visulise model
plt.figure(dpi=100)
visualize_classifier(X, y, forest)
plt.show()

# make predictions with test data
test_y_pred = forest.predict(test_X)

# compute accuarcy score with test data
print('Accuracy score on test data: %.2f' % accuracy_score(test_y, test_y_pred))

In [ ]:
# create and fit a AdaBoost classifier
ada = AdaBoostClassifier(n_estimators=300, learning_rate=.003, random_state=0).fit(X, y)

# visulise model
plt.figure(dpi=100)
visualize_classifier(X, y, ada)
plt.show()

# make predictions with test data
test_y_pred = ada.predict(test_X)

# compute accuarcy score with test data
print('Accuracy score on test data: %.2f' % accuracy_score(test_y, test_y_pred))

---

## Exercises 

There are many standard "toy" datasets for practicing classification with machine learning, such as `iris`, `wine` and `breast_cancer`. Many of them can be directly loaded from `sklearn`. Try to solve one or some of these problems using decision trees (either single or ensemble). Among these datasets, `iris` is the simplest one with the fewest features and classes.

In [ ]:
# load iris dataset
iris = sklearn.datasets.load_iris()
print(iris['DESCR'])